In [2]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from src.config import get_parser
from src.util import make_faces
from pytorch3d.structures import Meshes

config = get_parser().parse_args(args=[])

blueprint = np.load(os.path.join(config.data_dir, config.blueprint)) 
points = torch.tensor(blueprint['points'])
normals = torch.tensor(blueprint['normals'])
points = F.interpolate(points, size=config.data_blueprint_size,
                               mode='bicubic', align_corners=True)
normals = F.interpolate(normals, size=config.data_blueprint_size, 
                        mode='bicubic', align_corners=True)
normals = F.normalize(normals,  p=2, dim=1)
points.shape, normals.shape

(torch.Size([1, 3, 640, 640]), torch.Size([1, 3, 640, 640]))

In [10]:
cos_sim = torch.cosine_similarity(normals, normals)
cos_sim.sum() / cos_sim.numel()

tensor(1.)

In [14]:
wt = 0.1
cos_sim = torch.cosine_similarity(normals, normals + wt * torch.randn_like(normals))
cos_sim.sum() / cos_sim.numel()

tensor(0.9900)

In [15]:
torch.dot(normals, normals)

RuntimeError: 1D tensors expected, but got 4D and 4D tensors

In [17]:
cos_sim = torch.cosine_similarity(normals, -(normals + wt * torch.randn_like(normals)))
cos_sim.sum() / cos_sim.numel()

tensor(-0.9900)

In [ ]:
torch.bmm(A.view(6, 1, 256), B.view(6, 256, 1)) 

In [21]:
norms = normals.reshape(1, 3, -1)

torch.bmm(norms, norms.permute(0, 2, 1))

tensor([[[ 2.2519e+05,  4.1372e+02, -1.8156e+02],
         [ 4.1372e+02,  8.0160e+04,  9.0691e+03],
         [-1.8156e+02,  9.0691e+03,  1.0425e+05]]])

In [20]:
norms.shape

torch.Size([1, 3, 409600])

In [22]:
for i in [-1., -0.5, 0., 0.5, 1.0]:
    print(i, -(i-1))

-1.0 2.0
-0.5 1.5
0.0 1.0
0.5 0.5
1.0 -0.0
